<script>
    function findAncestor (el, name) {
        while ((el = el.parentElement) && el.nodeName.toLowerCase() !== name);
        return el;
    }
    function colorAll(el, textColor) {
        el.style.color = textColor;
        Array.from(el.children).forEach((e) => {colorAll(e, textColor);});
    }
    function setBackgroundImage(src, textColor) {
        var section = findAncestor(document.currentScript, "section");
        if (section) {
            section.setAttribute("data-background-image", src);
			if (textColor) colorAll(section, textColor);
        }
    }
</script>


# Starwars exploration with LDA

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import tree
from sklearn.tree import _tree
import pydotplus
from IPython.display import Image, display, clear_output
from ipywidgets import widgets, Layout
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier
from bokeh.plotting import *
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC, LinearSVC
from tabulate import tabulate
import squarify
import spacy
import string
from nltk.classify import SklearnClassifier
from subprocess import check_output
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import string
import base64
import re
from collections import Counter
from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
import holoviews as hv
import copy
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from time import time
stopwords = stopwords.words('english')
sns.set_context('talk')

import re
#spacy.load('en')
from spacy.lang.en import English
parser = English()
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfTransformer
import tabulate
from sklearn.metrics import confusion_matrix
from spacy import displacy

In [2]:
episodes= ["SW_EpisodeIV.txt","SW_EpisodeV.txt", "SW_EpisodeVI.txt"]
SW= []
star_all = pd.DataFrame()
for episode in episodes:
    star_1 = pd.read_csv(episode, sep='" "', engine='python')
    star_1 = star_1.loc[:,star_1.dtypes==object].apply(lambda star_1:star_1.str.replace('"', ""))
    star_1 = star_1.reset_index(drop=True)
    star_1.columns = ['character', 'dialogue']
    SW.append(star_1)
star_all = pd.concat(SW)
star_all = star_all.reset_index(drop=True)
star_all.shape

(2523, 2)

In [3]:
classes=star_all.character.value_counts()
classes_big=classes[classes>=40]
sufficient=star_all[star_all.character.isin(classes_big.index)][["character",'dialogue']]

In [4]:
sufficient=sufficient.reset_index(drop=True)
sufficient.shape

(1930, 2)

In [5]:
sufficient["tokens"]=sufficient['dialogue'].apply(word_tokenize) 

In [6]:
def no_punct(tokens):
    words = [word for word in tokens if word.isalpha()]
    return words
def to_lower(tokens):
    words = [w.lower() for w in tokens]
    return words

In [7]:
sufficient["tokens"]=sufficient['tokens'].apply(to_lower)
sufficient["tokens"]=sufficient['tokens'].apply(no_punct)

# lda try

In [8]:
sufficient.head()

,character,dialogue,tokens
0,THREEPIO,Did you hear that? They've shut down the main...,"[did, you, hear, that, they, shut, down, the, ..."
1,THREEPIO,We're doomed!,"[we, doomed]"
2,THREEPIO,There'll be no escape for the Princess this time.,"[there, be, no, escape, for, the, princess, th..."
3,THREEPIO,What's that?,"[what, that]"
4,THREEPIO,I should have known better than to trust the l...,"[i, should, have, known, better, than, to, tru..."


In [9]:
suff_grouped=sufficient.groupby('character')['dialogue'].apply(list)

In [10]:
suff_grouped2=sufficient.groupby('character')['tokens'].apply(list)
suff_grouped2

character
BEN         [[hello, there, come, here, my, little, friend...
EMPEROR     [[there, is, a, great, disturbance, in, the, f...
HAN         [[han, solo, i, captain, of, the, millennium, ...
LANDO       [[why, you, slimy, swindler, you, got, a, lot,...
LEIA        [[lord, vader, i, should, have, known, only, y...
LUKE        [[hurry, up, come, with, me, what, are, you, w...
THREEPIO    [[did, you, hear, that, they, shut, down, the,...
VADER       [[where, are, those, transmissions, you, inter...
YODA        [[hmmm, much, anger, in, him, like, his, fathe...
Name: tokens, dtype: object

In [11]:
from itertools import chain

In [12]:
superlist=(list(chain.from_iterable(suff_grouped2[0])), 
           list(chain.from_iterable(suff_grouped2[1])),
           list(chain.from_iterable(suff_grouped2[2])), 
           list(chain.from_iterable(suff_grouped2[3])), 
           list(chain.from_iterable(suff_grouped2[4])),
           list(chain.from_iterable(suff_grouped2[5])), 
           list(chain.from_iterable(suff_grouped2[6])),
           list(chain.from_iterable(suff_grouped2[7])),
           list(chain.from_iterable(suff_grouped2[8])))

In [15]:
import spacy
#spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [16]:
import nltk
#nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [17]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nataliespeiser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    #tokens = [token for token in tokens if token is not " "]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [19]:
flattened_list=[]
for x in superlist: #
    for y in x:
        flattened_list.append(y)

In [20]:
suff_grouped_list=suff_grouped[0]+suff_grouped[1]+suff_grouped[2]+suff_grouped[3]+suff_grouped[4]+suff_grouped[5]+suff_grouped[6]+suff_grouped[7]+suff_grouped[8]

In [21]:
superlist_new=[]
for characterlist in superlist:
    for token in characterlist:
        if token in en_stop:
            characterlist.remove(token)
    superlist_new.append(characterlist)

In [22]:
superlist_new=[]
characterlist_new=[]
for characterlist in superlist:
    characterlist_new=[token for token in characterlist if token not in en_stop]
    superlist_new.append(characterlist_new)

In [25]:
from gensim import corpora
dictionary = corpora.Dictionary(superlist_new)
corpus = [dictionary.doc2bow(text) for text in superlist_new]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [26]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [36]:
import gensim
NUM_TOPICS = 9
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * log

//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * log

//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * log

(0, '0.017*"young" + 0.015*"friends" + 0.010*"skywalker" + 0.009*"side"')
(1, '0.038*"oh" + 0.029*"sir" + 0.028*"artoo" + 0.014*"master"')
(2, '0.001*"luke" + 0.001*"oh" + 0.001*"artoo" + 0.001*"get"')
(3, '0.015*"right" + 0.011*"get" + 0.010*"vader" + 0.009*"well"')
(4, '0.021*"luke" + 0.010*"father" + 0.010*"know" + 0.009*"get"')
(5, '0.017*"get" + 0.016*"going" + 0.014*"right" + 0.013*"come"')
(6, '0.001*"come" + 0.001*"get" + 0.001*"right" + 0.001*"going"')
(7, '0.018*"force" + 0.016*"yes" + 0.014*"must" + 0.010*"dark"')
(8, '0.001*"going" + 0.001*"oh" + 0.001*"get" + 0.001*"right"')


//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
//anaconda/envs/starwars/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * log

In [52]:
new_doc = 'let us go kid we need to save our people'#spoken conversational exchange between two or more people, and a literary and theatrical form that depicts such an exchange. As a narrative, philosophical or didactic device, it is chiefly associated in the West with the Socratic dialogue as developed by Plato, but antecedents are also found in other traditions including Indian '
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(294, 1)]
[(0, 0.05555556), (1, 0.055555556), (2, 0.055555567), (3, 0.05557838), (4, 0.5554891), (5, 0.05559916), (6, 0.055555567), (7, 0.05555556), (8, 0.055555567)]


In [38]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)


//anaconda/envs/starwars/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


# What is the meaning of this super cool visualization?
With Ben selected there are two Bubbles getting big. Also there is just Luke and Leia using the word "ben" so I guess the Bubbles stand for those two characters (Threepio used the word bend, not Ben). <br> also this is the same case with "kid" it is just used by Han so just one of the Bubbles getting big. But there are just 6 bigger bubbles although we have 9 characters and 9 topics. Is it a valid statement to say, that the characters are not very good to differentiate?

In [48]:
sufficient[sufficient.tokens.astype(str).str.contains("ben")]

,character,dialogue,tokens
112,LUKE,Obi-Wan Kenobi? I wonder if he means old Ben ...,"[kenobi, i, wonder, if, he, means, old, ben, k..."
114,LUKE,"Well, I don't know anyone named Obi-Wan, but o...","[well, i, do, know, anyone, named, but, old, b..."
129,LUKE,I thought he might have meant Ben. Do you kno...,"[i, thought, he, might, have, meant, ben, do, ..."
148,LUKE,Old Ben Kenobi lives out in this direction som...,"[old, ben, kenobi, lives, out, in, this, direc..."
166,LUKE,"Ben? Ben Kenobi! Boy, am I glad to see you!","[ben, ben, kenobi, boy, am, i, glad, to, see, ..."
407,LUKE,Ben is a great man.,"[ben, is, a, great, man]"
467,LUKE,I'm here to rescue you. I've got your R2 unit...,"[i, here, to, rescue, you, i, got, your, unit,..."
468,LEIA,Ben Kenobi is here! Where is he?,"[ben, kenobi, is, here, where, is, he]"
643,LUKE,I only wish Ben were here.,"[i, only, wish, ben, were, here]"
772,LUKE,Ben?,[ben]


In [50]:
sufficient[sufficient.tokens.astype(str).str.contains("kid")]

,character,dialogue,tokens
50,LUKE,Join the Rebellion?! Are you kidding! How?,"[join, the, rebellion, are, you, kidding, how]"
260,HAN,"But who's going to fly it, kid! You?","[but, who, going, to, fly, it, kid, you]"
292,HAN,She'll make point five beyond the speed of lig...,"[she, make, point, five, beyond, the, speed, o..."
300,HAN,"Watch your mouth, kid, or you're going to find...","[watch, your, mouth, kid, or, you, going, to, ..."
304,LUKE,Are you kidding? At the rate they're gaining...,"[are, you, kidding, at, the, rate, they, gaining]"
332,HAN,Hokey religions and ancient weapons are no mat...,"[hokey, religions, and, ancient, weapons, are,..."
334,HAN,"Kid, I've flown from one side of this galaxy t...","[kid, i, flown, from, one, side, of, this, gal..."
354,HAN,"Thats what I'm trying to tell you, kid. It ai...","[thats, what, i, trying, to, tell, you, kid, i..."
381,HAN,"There's nothin' I can do about it, kid. I'm i...","[there, nothin, i, can, do, about, it, kid, i,..."
440,HAN,"All right, kid. But you'd better be right abo...","[all, right, kid, but, you, better, be, right,..."
